In [1]:
import pandas as pd
import json
import seaborn as sns
import networkx as nx
import itertools

In [27]:
with open('train.jsonl') as f:
    df_raw = pd.DataFrame(json.loads(line) for line in f)
df_raw.drop('Unnamed: 0', axis=1, inplace=True)
df = df_raw.drop(df_raw[df_raw['condition'].str.contains("<", na=False)].index)
print(f"removed {df_raw.shape[0]-df.shape[0]} rows")
df

removed 900 rows


,drugName,condition,review,rating,date,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37
...,...,...,...,...,...,...
161292,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10.0,"May 31, 2015",125
161293,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0,"November 1, 2011",34
161294,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2.0,"March 15, 2014",35
161295,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10.0,"September 19, 2015",79


In [29]:
ranked_drugs = df['drugName'].value_counts()
conditions = df['condition'].value_counts()
# filter above certain threshhold 
#ranked_drugs = ranked_drugs.loc[lambda x: x>100]

In [30]:
ranked_drugs.head()

Levonorgestrel                       3657
Etonogestrel                         3336
Ethinyl estradiol / norethindrone    2850
Nexplanon                            2149
Ethinyl estradiol / norgestimate     2117
Name: drugName, dtype: int64

In [36]:
# Takes a while
edge_list = []
print("Progress:")
for drug, drug_count in ranked_drugs.items():
    # all of the conditions per drug
    conds_series = df.loc[df['drugName'] == drug]['condition'].value_counts()
    
    for cond, cond_count in conds_series.items():
        # filter df per condition to get df with only the same condition and drug in the loop
        
        filtered_df = df.loc[(df['drugName'] == drug) & (df['condition'] == cond)]
        sum = filtered_df['rating'].sum()
        avg = sum / filtered_df.shape[0]
        
        edge_list.append((drug, cond, cond_count, avg))
    print(".", end='')
print("Done!")

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [40]:
import json
graph_data = {
    'drug' : ranked_drugs.axes[0].values.tolist(),
    'condition' : conditions.axes[0].values.tolist(),
    'edges' : edge_list
}
with open('graph_dict.json', 'w') as f:
    f.write(json.dumps(graph_data))